# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.

In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

In [5]:
df = pd.read_pickle(r"shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


In [6]:
df1 = df.dropna(subset = ['BOROUGH'])
df1

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
7,2021-12-14 08:17:00,BRONX,10475.0,40.868160,-73.831480,"(40.86816, -73.83148)",NaN,NaN,344 BAYCHESTER AVENUE,2.0,...,Unspecified,NaN,NaN,NaN,4486660,Sedan,Sedan,NaN,NaN,NaN
8,2021-12-14 21:10:00,BROOKLYN,11207.0,40.671720,-73.897100,"(40.67172, -73.8971)",NaN,NaN,2047 PITKIN AVENUE,0.0,...,Unspecified,NaN,NaN,NaN,4487074,Sedan,NaN,NaN,NaN,NaN
9,2021-12-14 14:58:00,MANHATTAN,10017.0,40.751440,-73.973970,"(40.75144, -73.97397)",3 AVENUE,EAST 43 STREET,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4486519,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018237,2023-07-13 21:30:00,MANHATTAN,10027.0,40.814323,-73.951180,"(40.814323, -73.95118)",NaN,NaN,401 WEST 130 STREET,0.0,...,NaN,NaN,NaN,NaN,4648334,Sedan,NaN,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


In [11]:
df1 = df1.copy()
df1['CRASH DATE_CRASH TIME'] = pd.to_datetime(df1['CRASH DATE_CRASH TIME'])
df1_sorted = df1.sort_values(by = 'CRASH DATE_CRASH TIME')

df1_sorted

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
1923982,2012-07-01 00:05:00,MANHATTAN,10036,40.762127,-73.997387,"(40.7621266, -73.9973865)",11 AVENUE,WEST 44 STREET,NaN,0.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,37632,PASSENGER VEHICLE,BUS,NaN,NaN,NaN
1923574,2012-07-01 00:10:00,BROOKLYN,11223,40.588868,-73.972745,"(40.5888678, -73.9727446)",WEST 3 STREET,BOUCK COURT,NaN,0.0,...,Unspecified,NaN,NaN,NaN,116256,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
1920559,2012-07-01 00:20:00,BROOKLYN,11215,40.677406,-73.983048,"(40.6774056, -73.9830482)",4 AVENUE,UNION STREET,NaN,0.0,...,Unspecified,NaN,NaN,NaN,175808,UNKNOWN,BICYCLE,NaN,NaN,NaN
1924529,2012-07-01 00:22:00,BRONX,10451,40.824188,-73.913840,"(40.8241884, -73.91384)",EAST 161 STREET,MELROSE AVENUE,NaN,0.0,...,Backing Unsafely,NaN,NaN,NaN,78654,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
1923369,2012-07-01 00:23:00,BROOKLYN,11234,40.607179,-73.916414,"(40.6071786, -73.9164142)",STRICKLAND AVENUE,NATIONAL DRIVE,NaN,0.0,...,Unspecified,NaN,NaN,NaN,125655,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1946217,2023-08-15 22:28:00,BROOKLYN,11210.0,40.627690,-73.935555,"(40.62769, -73.935555)",AVENUE J,EAST 42 STREET,NaN,3.0,...,Aggressive Driving/Road Rage,Other Vehicular,NaN,NaN,4654582,Sedan,Sedan,Sedan,NaN,NaN
1946380,2023-08-15 23:30:00,QUEENS,11369.0,40.768955,-73.868600,"(40.768955, -73.8686)",DITMARS BOULEVARD,102 STREET,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4654488,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1946683,2023-08-15 23:44:00,QUEENS,11432.0,40.707237,-73.792710,"(40.707237, -73.79271)",NaN,NaN,168-02 91 AVENUE,0.0,...,Unspecified,NaN,NaN,NaN,4654431,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN
1946758,2023-08-15 23:55:00,BRONX,10472.0,40.832943,-73.872430,"(40.832943, -73.87243)",NaN,NaN,1329 FTELEY AVENUE,0.0,...,Unspecified,NaN,NaN,NaN,4654897,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


Q1

In [12]:
borough_counts = df1_sorted['BOROUGH'].value_counts()

print(borough_counts)

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: count, dtype: int64


Q2

In [ ]:
populations = {
    'BRONX': 1446788,
    'BROOKLYN': 2648452,
    'MANHATTAN': 1638281,
    'QUEENS': 2330295,
    'STATEN ISLAND': 487155
}

In [18]:
crashes_per_100k = {}
for borough, count in borough_counts.items():
    crashes_per_100k[borough] = (count / populations[borough]) * 100000

# Sort the boroughs by crashes per 100,000 people in descending order
sorted_boroughs = sorted(crashes_per_100k.items(), key=lambda x: x[1], reverse=True)

# Display the results
for borough, value in sorted_boroughs:
    print(f"{borough}: {value:.2f} crashes per 100,000 people")

# To get the borough with the highest crashes per 100,000
print(f"\n{sorted_boroughs[0][0]} has the highest number of crashes per 100,000 people.")    

MANHATTAN: 19121.63 crashes per 100,000 people
BROOKLYN: 16652.22 crashes per 100,000 people
QUEENS: 15983.26 crashes per 100,000 people
BRONX: 14193.16 crashes per 100,000 people
STATEN ISLAND: 11966.83 crashes per 100,000 people

MANHATTAN has the highest number of crashes per 100,000 people.


Q3

In [19]:
df_filtered = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']
factor_counts = df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# Determine the leading cause of crashes
leading_cause = factor_counts.idxmax()
leading_cause_count = factor_counts.max()

# Calculate the proportion of accidents attributable to the leading cause
proportion = leading_cause_count / df_filtered.shape[0]

print(f"The leading cause of crashes is: {leading_cause}")
print(f"This cause is responsible for {proportion:.2%} of the accidents.")

The leading cause of crashes is: Driver Inattention/Distraction
This cause is responsible for 30.27% of the accidents.


Q4

In [23]:
# Sum the counts of the top 3 causes
top_3_causes_count = factor_counts.head(3).sum()

# Calculate the proportion of accidents attributable to the top 3 causes
proportion_top_3 = top_3_causes_count / df_filtered.shape[0]

print(f"The top 3 causes of crashes account for {proportion_top_3:.2%} of the accidents.")

The top 3 causes of crashes account for 47.37% of the accidents.


Q5

In [24]:
df.columns

Index(['CRASH DATE_CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'COLLISION_ID', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')

In [26]:
fatal_crashes = df[
    (df['NUMBER OF PERSONS KILLED'] > 0)]

# Count the number of rows in the filtered dataset
fatal_count = fatal_crashes.shape[0]

print(f"Total count of accidents that involved at least one fatality: {fatal_count}")

Total count of accidents that involved at least one fatality: 2804


Q6

In [27]:
total_accidents = df.shape[0]

Fatal_Accidents_per_1000 = (fatal_count / total_accidents)*1000

Fatal_Accidents_per_1000

1.3893258747079764

Q7

In [32]:
missing_borough_count = df['BOROUGH'].isnull().sum()

proportion_missing_borough = missing_borough_count/df.shape[0]

{proportion_missing_borough * 100}

SyntaxError: invalid syntax (1356994266.py, line 5)

Q8

In [ ]:
# Group by 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2', then count the occurrences
vehicle_combinations = df.groupby(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).size()

# Find the combination with the most occurrences
most_common_combination = vehicle_combinations.idxmax()
most_common_count = vehicle_combinations.max()

print(f"The most common combination of vehicles is {most_common_combination} with {most_common_count} accidents.")